In [1]:
import math
from multiprocessing import Pool
import os
import sys
import itertools
import time
import tempfile
import pickle

sys.path.append(os.path.abspath(os.pardir))

%matplotlib inline

import keras
import numpy as np
import pandas as pd
import seaborn as sns

# Metrics
from sklearn.metrics import accuracy_score

# Notebook helper methods
from tdparse import notebook_helper
# Models
from tdparse.models.tdlstm import LSTM, TDLSTM, TCLSTM
# Tokenisers
from tdparse.tokenisers import ark_twokenize
# Word Vectors
from tdparse.word_vectors import PreTrained, GloveCommonCrawl
# Get the data
from tdparse.parsers import semeval_14, dong, election
from tdparse.data_types import TargetCollection
from tdparse.helper import read_config, full_path
from tdparse.evaluation import evaluation_results
from tdparse.notebook_helper import get_json_data, write_json_data

Using TensorFlow backend.


In [2]:
# Load all of the datasets
youtubean_train = semeval_14(full_path(read_config('youtubean_train')))
youtubean_test = semeval_14(full_path(read_config('youtubean_test')))
semeval_14_rest_train = semeval_14(full_path(read_config('semeval_2014_rest_train')))
semeval_14_lap_train = semeval_14(full_path(read_config('semeval_2014_lap_train')))
semeval_14_rest_test = semeval_14(full_path(read_config('semeval_2014_rest_test')))
semeval_14_lap_test = semeval_14(full_path(read_config('semeval_2014_lap_test')))
dong_train = dong(full_path(read_config('dong_twit_train_data')))
dong_test = dong(full_path(read_config('dong_twit_test_data')))
election_train, election_test = election(full_path(read_config('election_folder_dir')))
mitchel_train = semeval_14(full_path(read_config('mitchel_train')))
mitchel_test = semeval_14(full_path(read_config('mitchel_test')))


dataset_train_test = {#'SemEval 14 Laptop' : (semeval_14_lap_train, semeval_14_lap_test),
                      #'SemEval 14 Restaurant' : (semeval_14_rest_train, semeval_14_rest_test),
                      #'Dong Twitter' : (dong_train, dong_test),
                      'Election Twitter' : (election_train, election_test),
                      #'YouTuBean' : (youtubean_train, youtubean_test),
                      #'Mitchel' : (mitchel_train, mitchel_test)
                     }

In [3]:
# Loading the word vectors
sswe_path = full_path(read_config('sswe_files')['vo_zhang'])
sswe = PreTrained(sswe_path, name='sswe')
glove_300 = GloveCommonCrawl(version=42)
# Word vectors that we are searching over
word_vectors = [sswe, glove_300]


# This is required as we have 3 classes and one of them is -1 and when one hot encoded
# the index of -1 is 2 and that is what it thinks the label is when it should be 
# -1 hence the sentiment mapper
sentiment_mapper = {0 : 0, 1 : 1, 2 : -1}

# Folder to store all the sub folder for each model
result_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'results', 'TDLstm'))
# Folder to store all of the saved models (model zoo folder)
model_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'model zoo'))
os.makedirs(model_dir, exist_ok=True)

In [4]:
def dataset_predictions(train, test, dataset_name, model_class, 
                        word_vector_file_path, result_file_path,
                        model_folder_path, model_params):
    
    print('{} {}'.format(dataset_name, model_params))

    data_train = train.data_dict()
    y_train = train.sentiment_data()
    data_test = test.data_dict()
    y_test = test.sentiment_data()
    
    # Fits the model
    word_vector_data = get_json_data(word_vector_file_path, dataset_name)
    best_score = 0
    best_word_vector = None
    best_model = None
    for word_vector in word_vectors:
        print(word_vector)
        word_vector_name = '{}'.format(word_vector)
        if word_vector_name in word_vector_data:
            word_vec_val_score = word_vector_data[word_vector_name]
            if word_vec_val_score > best_score:
                best_score = word_vec_val_score
                best_word_vector = word_vector
            continue
        model_params['embeddings'] = word_vector
        model = model_class(**model_params)
        print('{} {}'.format(model_params, word_vector))
        history = model.fit(data_train, y_train, validation_size=0.3, verbose=1,
                            reproducible=True, patience=10, epochs=300, org_initialisers=True)
        word_vec_val_score = max(history.history['val_acc'])
        word_vector_data[word_vector_name] = word_vec_val_score
        if word_vec_val_score > best_score:
                best_score = word_vec_val_score
                best_word_vector = word_vector
                best_model = model
                
        # Save word vector validation score result
        write_json_data(word_vector_file_path, dataset_name, word_vector_data)
    if best_word_vector is None:
        raise ValueError('best word vector should not be None')
    if best_model is None:
        model_params['embeddings'] = best_word_vector
        model = model_class(**model_params)
        print('{} {}'.format(model_params, best_word_vector))
        model.fit(data_train, y_train, validation_size=0.3, verbose=1,
                  reproducible=True, patience=10, epochs=300, org_initialisers=True)
    # Saves the model to the model zoo
    model_folder_join = lambda file_name: os.path.join(model_folder_path, file_name)
    model_arch_fp = model_folder_join('{} {} architecture'.format(model, dataset_name))
    model_weights_fp = model_folder_join('{} {} weights'.format(model, dataset_name))
    model.save_model(model_arch_fp, model_weights_fp, verbose=1)
    
    # Predicts on the test data
    predicted_values = model.predict(data_test)
    # Convert prediction from one hot encoded to category value e.g. -1, 0, 1
    predicted_values_cats =  model.prediction_to_cats(y_test, predicted_values, 
                                                      mapper=sentiment_mapper)
    # Evaluates the predictions and save the results
    return evaluation_results(predicted_values_cats, test, dataset_name, 
                              file_name=result_file_path, 
                              save_raw_data=True, re_write=True)

# Mass Evaluation of the LSTM model

In [5]:
# Model folder results
lstm_folder = os.path.join(result_folder, 'lstm')
os.makedirs(lstm_folder, exist_ok=True)

# Result files
word_vector_file = os.path.join(lstm_folder, 'word vector results.json')
result_file = os.path.join(lstm_folder, 'results file.tsv')

for dataset_name, train_test in dataset_train_test.items():
    train, test = train_test
    model_params = {'tokeniser' : ark_twokenize,
                    'lower' : True, 'pad_size' : -1}
    dataset_predictions(train, test, dataset_name, LSTM, 
                        word_vector_file, result_file, model_dir, model_params)

Election Twitter {'tokeniser': <function ark_twokenize at 0x7fd7b4194950>, 'lower': True, 'pad_size': -1}
sswe
glove 300d 42b common crawl
{'tokeniser': <function ark_twokenize at 0x7fd7b4194950>, 'lower': True, 'pad_size': -1, 'embeddings': glove 300d 42b common crawl} glove 300d 42b common crawl
Train on 6550 samples, validate on 2808 samples
Epoch 1/300
6550/6550 [==============================] - 84s 13ms/step - loss: 1.0603 - acc: 0.4655 - val_loss: 1.0343 - val_acc: 0.4676
Epoch 2/300
6550/6550 [==============================] - 87s 13ms/step - loss: 1.0225 - acc: 0.4678 - val_loss: 1.0141 - val_acc: 0.4676
Epoch 3/300
6550/6550 [==============================] - 85s 13ms/step - loss: 1.0100 - acc: 0.4678 - val_loss: 1.0070 - val_acc: 0.4676
Epoch 4/300
6550/6550 [==============================] - 88s 13ms/step - loss: 1.0056 - acc: 0.4678 - val_loss: 1.0045 - val_acc: 0.4676
Epoch 5/300
6550/6550 [==============================] - 90s 14ms/step - loss: 1.0041 - acc: 0.4678 - val

Epoch 58/300
6550/6550 [==============================] - 85s 13ms/step - loss: 0.9758 - acc: 0.5301 - val_loss: 0.9884 - val_acc: 0.5046
Epoch 59/300
6550/6550 [==============================] - 88s 13ms/step - loss: 0.9737 - acc: 0.5311 - val_loss: 0.9834 - val_acc: 0.5014
Epoch 60/300
6550/6550 [==============================] - 86s 13ms/step - loss: 0.9715 - acc: 0.5307 - val_loss: 0.9818 - val_acc: 0.5000
Epoch 61/300
6550/6550 [==============================] - 81s 12ms/step - loss: 0.9698 - acc: 0.5321 - val_loss: 0.9803 - val_acc: 0.5039
Epoch 62/300
6550/6550 [==============================] - 78s 12ms/step - loss: 0.9669 - acc: 0.5337 - val_loss: 0.9845 - val_acc: 0.5157
Epoch 63/300
6550/6550 [==============================] - 80s 12ms/step - loss: 0.9644 - acc: 0.5394 - val_loss: 0.9783 - val_acc: 0.5139
Epoch 64/300
6550/6550 [==============================] - 87s 13ms/step - loss: 0.9639 - acc: 0.5316 - val_loss: 0.9774 - val_acc: 0.5207
Epoch 65/300
6550/6550 [==========

6550/6550 [==============================] - 83s 13ms/step - loss: 0.9178 - acc: 0.5710 - val_loss: 0.9487 - val_acc: 0.5402
Epoch 118/300
6550/6550 [==============================] - 79s 12ms/step - loss: 0.9197 - acc: 0.5705 - val_loss: 0.9854 - val_acc: 0.4900
Epoch 119/300
6550/6550 [==============================] - 78s 12ms/step - loss: 0.9219 - acc: 0.5632 - val_loss: 0.9589 - val_acc: 0.5199
Epoch 120/300
6550/6550 [==============================] - 79s 12ms/step - loss: 0.9215 - acc: 0.5647 - val_loss: 0.9508 - val_acc: 0.5449
Epoch 121/300
6550/6550 [==============================] - 83s 13ms/step - loss: 0.9178 - acc: 0.5704 - val_loss: 0.9472 - val_acc: 0.5442
Epoch 122/300
6550/6550 [==============================] - 79s 12ms/step - loss: 0.9186 - acc: 0.5676 - val_loss: 0.9496 - val_acc: 0.5324
Epoch 123/300
6550/6550 [==============================] - 79s 12ms/step - loss: 0.9158 - acc: 0.5693 - val_loss: 0.9551 - val_acc: 0.5274
Epoch 124/300
6550/6550 [================

# Mass Evaluation of the TDLSTM model

In [5]:
# Model folder results
tdlstm_folder = os.path.join(result_folder, 'tdlstm')
os.makedirs(tdlstm_folder, exist_ok=True)

# Result files
word_vector_file = os.path.join(tdlstm_folder, 'word vector results.json')
result_file = os.path.join(tdlstm_folder, 'results file.tsv')

for dataset_name, train_test in dataset_train_test.items():
    train, test = train_test
    model_params = {'tokeniser' : ark_twokenize,
                    'lower' : True, 'pad_size' : -1}
    dataset_predictions(train, test, dataset_name, TDLSTM,
                        word_vector_file, result_file, model_dir, model_params)

Mitchel {'tokeniser': <function ark_twokenize at 0x7f2c83bdd950>, 'lower': True, 'pad_size': -1}
sswe
glove 300d 42b common crawl
{'tokeniser': <function ark_twokenize at 0x7f2c83bdd950>, 'lower': True, 'pad_size': -1, 'embeddings': glove 300d 42b common crawl} glove 300d 42b common crawl
Train on 1610 samples, validate on 691 samples
Epoch 1/5
1610/1610 [==============================] - 46s 29ms/step - loss: 0.9869 - acc: 0.6957 - val_loss: 0.9009 - val_acc: 0.7019
Epoch 2/5
1610/1610 [==============================] - 48s 30ms/step - loss: 0.8628 - acc: 0.7012 - val_loss: 0.8373 - val_acc: 0.7019
Epoch 3/5
1610/1610 [==============================] - 48s 30ms/step - loss: 0.8251 - acc: 0.7012 - val_loss: 0.8170 - val_acc: 0.7019
Epoch 4/5
1610/1610 [==============================] - 49s 30ms/step - loss: 0.8125 - acc: 0.7012 - val_loss: 0.8097 - val_acc: 0.7019
Epoch 5/5
1610/1610 [==============================] - 47s 29ms/step - loss: 0.8064 - acc: 0.7012 - val_loss: 0.8056 - val_

KeyboardInterrupt: 

# Mass Evaluation of the TCLSTM model

In [ ]:
# Model folder results
tclstm_folder = os.path.join(result_folder, 'tclstm')
os.makedirs(tclstm_folder, exist_ok=True)

# Result files
word_vector_file = os.path.join(tclstm_folder, 'word vector results.json')
result_file = os.path.join(tclstm_folder, 'results file.tsv')

for dataset_name, train_test in dataset_train_test.items():
    train, test = train_test
    model_params = {'tokeniser' : ark_twokenize,
                    'lower' : True, 'pad_size' : -1}
    dataset_predictions(train, test, dataset_name, TCLSTM, 
                        word_vector_file, result_file, model_dir, model_params)

Mitchel {'tokeniser': <function ark_twokenize at 0x7fbd07a14950>, 'lower': True, 'pad_size': -1}
sswe
{'tokeniser': <function ark_twokenize at 0x7fbd07a14950>, 'lower': True, 'pad_size': -1, 'embeddings': sswe} sswe
Train on 1610 samples, validate on 691 samples
Epoch 1/300
1610/1610 [==============================] - 6s 4ms/step - loss: 0.8468 - acc: 0.7006 - val_loss: 0.7888 - val_acc: 0.7019
Epoch 2/300
1610/1610 [==============================] - 6s 4ms/step - loss: 0.7847 - acc: 0.7012 - val_loss: 0.7809 - val_acc: 0.7019
Epoch 3/300
1610/1610 [==============================] - 6s 4ms/step - loss: 0.7818 - acc: 0.7012 - val_loss: 0.7805 - val_acc: 0.7019
Epoch 4/300
1610/1610 [==============================] - 6s 4ms/step - loss: 0.7814 - acc: 0.7012 - val_loss: 0.7798 - val_acc: 0.7019
Epoch 5/300
1610/1610 [==============================] - 6s 4ms/step - loss: 0.7800 - acc: 0.7012 - val_loss: 0.7791 - val_acc: 0.7019
Epoch 6/300
1610/1610 [==============================] - 6s 4m

1610/1610 [==============================] - 147s 91ms/step - loss: 0.7314 - acc: 0.7062 - val_loss: 0.7529 - val_acc: 0.7019
Epoch 25/300
1610/1610 [==============================] - 151s 94ms/step - loss: 0.7279 - acc: 0.7087 - val_loss: 0.7509 - val_acc: 0.7019
Epoch 26/300
1408/1610 [=========================>....] - ETA: 13s - loss: 0.7263 - acc: 0.7067